In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt


# Тут идет калибровка на шахматную доску

chessboard_size = (10, 7) 

objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)


objpoints = []  
imgpoints = [] 

image_dir = 'ChessImages'
images = [img for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]

for image_name in images:
    img = cv2.imread(os.path.join(image_dir, image_name))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)


        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [2]:
#Рисуем пирамиды

def draw_pyramid(img, base_center, base_size, height, rvecs, tvecs, camera_matrix, dist_coeffs):

    # Координаты вершин основания (квадрат)
    half_size = base_size / 2
    base_points = np.array([
        [base_center[0] - half_size, base_center[1] - half_size, base_center[2]],
        [base_center[0] + half_size, base_center[1] - half_size, base_center[2]],
        [base_center[0] + half_size, base_center[1] + half_size, base_center[2]],
        [base_center[0] - half_size, base_center[1] + half_size, base_center[2]],
    ])

    # Вершина пирамиды
    apex = np.array([[base_center[0], base_center[1], base_center[2] + height]])
    pyramid_points = np.vstack([base_points, apex])

    # Проекция точек пирамиды
    imgpts, jac = cv2.projectPoints(pyramid_points, rvecs, tvecs, camera_matrix, dist_coeffs)
    imgpts = imgpts.reshape(-1, 2).astype(int)

    # основание (квадрат)
    for i in range(4):
        cv2.line(img, tuple(imgpts[i]), tuple(imgpts[(i + 1) % 4]), (0, 0, 0), 2)

    # ребра от основания к вершине
    for i in range(4):
        cv2.line(img, tuple(imgpts[i]), tuple(imgpts[4]), (0, 0, 0), 2)

    return img

circle_coords = np.array([
    [449, 782],
    [519, 775],
    [599, 765],
    [449, 693],
    [514, 678],
    [594, 663],
    [449, 607],
    [514, 589],
    [590, 566]
], dtype=np.float32)

objp = np.zeros((3 * 3, 3), np.float32)
objp[:, :2] = np.mgrid[0:3, 0:3].T.reshape(-1, 2)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
retval, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, circle_coords, camera_matrix, dist_coeffs)

image_path = 'Map1_agles.jpg'
img = cv2.imread(image_path)

# Список центров пирамид отн-но НК
base_centers = [
    [-0.5, 0.2, 0],
    [0.0, -0.3, -3.0],
    [-0.5, -0.8, 0],
    [0.0, -0.8, -1.7]
]
base_size = 0.4 
height = 1.0 

# Отрисовка пирамид
for base_center in base_centers:
    img = draw_pyramid(img, base_center, base_size, height, rvecs, tvecs, camera_matrix, dist_coeffs)


cv2.imshow('Pyramids with Axes on Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


